In [1]:
import pandas as pd
import numpy as np
import time
import requests
import os

**The data scraped is from basketball reference so credit to them. Additionally, the players found in these tables needed to have played 75% of their team's games that season. It should be noted that 2011 is the first year that basketball reference includes BPM for college basketball players.**

In [2]:
#This function takes a start_year and end_year and retrieves advanced statistics for all college basketball player
#seasons in that span among those players that played 75% of their team's games that season. These are compiled and 
#downloaded to a file called "college_advanced.csv" as the final result

def get_advanced(start_year=2011, end_year=2022, final_df = pd.DataFrame()):
    #basketball reference only shows 100 players at a time from "offset" to "offset" + 100
    #I don't know the max "offset" value, but I do know it goes up by 100 so we'll loop by 100 from 0 to 1000000 (a number that is easily large enough to capture all player)
    #There is no way there were 1 million players from 2011 to 2022
    for i in range(0, 1000000, 100):
        
        #test just 5 loops to start to ensure things go smoothly
        #if i==500:
        #    break
         
        try: #eventually we will get all the players and will reach an "offset" that gives a table that doesn't exist
            url = f"https://www.sports-reference.com/cbb/play-index/psl_finder.cgi?request=1&match=single&year_min={start_year}&year_max={end_year}&conf_id=&school_id=&class_is_fr=Y&class_is_so=Y&class_is_jr=Y&class_is_sr=Y&pos_is_g=Y&pos_is_f=Y&pos_is_c=Y&games_type=A&qual=pts_per_g&c1stat=&c1comp=gt&c1val=&c2stat=&c2comp=gt&c2val=&c3stat=&c3comp=gt&c3val=&c4stat=&c4comp=gt&c4val=&order_by=bpm&order_by_asc=&offset={i}"
            print("Visiting: " + url) #use for debugging
            page = requests.get(url)
            page.raise_for_status()
            # get the table with player stats
            df = pd.read_html(page.text)[0]
            # returns multi-index dataframe with level 0 being which table and level 1 being the column names
            # we just want the column names so we need to drop the level 0 column header
            df = df.droplevel(level=0, axis=1)
            #drop na rows that come from repeated header level 0 (which table)
            df = df.dropna()
            #there are several repeated column header rows, the lines below find those row indices and then drop them
            repeated_header_indices = list(df[df.Player == "Player"].index)
            df = df.drop(repeated_header_indices).reset_index(drop=True)
            #append this table to the final table with all player from start_year to end_year
            final_df = final_df.append(df)
            #take 2 seconds between each loop to be curteous and not bombard the website
            time.sleep(2)
                 
        except ValueError: #when table doesn't exist a value error will be thrown, break the loop as we have everything
            print("No more player data to acquire, breaking the loop")
            break
            
    #reset the index of the final dataframe
    final_df = final_df.reset_index(drop=True)
    #the below lines write the dataframe to a file called filename
    filename = "college_advanced.csv"
    if not os.path.exists(filename):
        final_df.to_csv(filename, index=False)
        return(f"Downloaded file with college players advanced stats from {start_year} to {end_year} to file named {filename}")
    else:
        return(f"File {filename} already exists")
    
        

In [3]:
#t1 = time.time()
#get_advanced()
#t2 = time.time()

In [4]:
#(t2-t1)/60 #took about 30 minutes to get all the data, might be useful to do this in parallel

**Season is the year of the NCAA Tournament. So a season of 2019 means it was the 2018-19 season. This is done so that it is easier to match up the college season with the year of the NBA combine.**

In [5]:
stats = pd.read_csv("college_advanced.csv") # read in data
#keep only selected columns
stats = stats[["Player", "Class", "Season", "Pos", "School", "Conf", "TS%", "eFG%", "ORB%", "DRB%", "TRB%", "AST%", 
              "STL%", "BLK%", "TOV%", "USG%", "PProd", "OWS", "DWS", "WS", "OBPM", "DBPM", "BPM"]]
#split the hyphenated season into start year and end year
stats[["Start_Year", "End_Year"]] = stats["Season"].str.split('-', expand=True)
#edit the season column so that it just keeps the end year (year of the NCAA tournament and the NBA combine)
stats["Season"] = (stats["Start_Year"].astype(int) + 1).astype(str)
#drop excess columns
stats = stats.drop(columns=["Start_Year", "End_Year"])
#filter from 2019 season and earlier because 2019 is the latest class with 3rd year NBA BPM
stats = stats[stats.Season.astype(int) <= 2019]
stats.head(10)

,Player,Class,Season,Pos,School,Conf,TS%,eFG%,ORB%,DRB%,...,BLK%,TOV%,USG%,PProd,OWS,DWS,WS,OBPM,DBPM,BPM
0,Zion Williamson,FR,2019,F,Duke,ACC,0.702,0.708,12.7,18.0,...,5.8,12.8,28.6,663,5.5,2.8,8.3,13.4,6.7,20.1
1,Anthony Davis,FR,2012,F,Kentucky,SEC,0.654,0.628,11.6,25.6,...,13.7,8.6,18.8,550,5.9,4.1,9.9,9.1,8.1,17.2
2,Sindarius Thornwell,SR,2017,G,South Carolina,SEC,0.590,0.510,8.0,16.3,...,3.4,11.9,29.5,634,4.4,2.8,7.2,11.0,6.2,17.1
3,Brandon Clarke,JR,2019,F,Gonzaga,WCC,0.699,0.693,13.9,19.6,...,11.3,11.3,23.9,603,5.5,3.2,8.8,9.8,6.5,16.3
4,Frank Kaminsky,SR,2015,C,Wisconsin,Big Ten,0.628,0.590,6.0,25.7,...,4.5,9.8,28.6,691,6.9,2.9,9.8,11.2,5.0,16.2
6,Denzel Valentine,SR,2016,G,Michigan State,Big Ten,0.608,0.579,3.0,21.2,...,0.7,14.8,28.5,622,5.0,2.2,7.2,11.3,4.3,15.6
7,Delon Wright,SR,2015,G,Utah,Pac-12,0.619,0.549,4.0,13.7,...,3.2,14.2,22.8,558,5.3,3.1,8.4,8.7,6.9,15.5
8,Victor Oladipo,JR,2013,G,Indiana,Big Ten,0.671,0.648,11.8,15.1,...,2.8,18.5,22.2,478,4.2,2.6,6.7,8.9,6.2,15.1
11,Gary Clark,SR,2018,F,Cincinnati,AAC,0.612,0.571,12.1,22.3,...,5.0,8.7,20.4,462,4.3,3.5,7.8,7.9,6.4,14.4
12,Ethan Happ,SO,2017,F,Wisconsin,Big Ten,0.576,0.586,13.2,25.4,...,4.7,15.1,26.7,532,3.4,3.0,6.4,7.6,6.7,14.3


**To combine player's college stats into one row for the final dataset, I will be taking a weighted average. For non-numerical data columns (like name, position, school, etc.) I will be taking the information from their most recent college season. For the numerical columns, I will be taking a weighted average of their seasons. The seasons are weighted so that each additional season is worth 2x the previous season. For example, if a player played 2 seasons, their second season would be weighted at 66.7% while their first season would be weighted at 33.3%. In the case of a 4 year player, the weights would be as followed: Season 1=6.66%, Season 2=13.33%, Season 3=26.66%, Season 4=53.33%. As you can see each additional year is worth twice as much as the previous. I wanted to weight the more recent seasons more heavily as they are more reflective of the player.**

In [6]:
def weighted_avg(df):
    #create an array of the length of the player dataframe (length 4 means 4 seasons)
    weights = np.ones(len(df))
    # loop to make each value twice as much as the previous
    for i in range(len(weights)):
        if i == 0:
            continue
        weights[i] = weights[i-1] * 2
    #get the corresponding weights as decimals
    weights = (weights * 100/sum(weights))/100
    
    weighted_player = [] #list that will have the weighted values for the player
    #loop through the columns
    for col in df.columns:
        if df[col].dtypes == "O": #if the column is non-numeric, take the most recent year's data
            value = df[col].iloc[len(df)-1]
        if df[col].dtypes == "int64" or df[col].dtypes == "float64": #if the column is numeric take the weighted average
            value = round(np.average(df[col], weights=weights), 3)
        weighted_player.append(value) #append value for each column to the final list
        
    return(weighted_player)

**There is a slight problem with the format of this dataset and that is that it is difficult to distinguish players with the same name. For example, there are 2 different Gary Clark's in this dataset. One played one year at Wake Forest in 2011 while the other played four years at Cincinnati from 2015-2018. The best way I can think of to distinguish between players is by splitting them by schools. There are two downsides with this: 1) if two players with the same name played at the same school from 2011 to 2019, the weighted stats are still useless as they are combining two different players 2) the same player who played at different schools (ie transfer players) will be considered as two different players.**

In [7]:
#split by school so that these two different Gary Clarks aren't combined into one weighted average
example1 = stats[stats["Player"] == "Gary Clark"].sort_values(by="Season")
example1 

,Player,Class,Season,Pos,School,Conf,TS%,eFG%,ORB%,DRB%,...,BLK%,TOV%,USG%,PProd,OWS,DWS,WS,OBPM,DBPM,BPM
9568,Gary Clark,SR,2011,G,Wake Forest,ACC,0.655,0.626,1.8,7.3,...,0.5,17.1,18.9,312,1.9,0.2,2.1,2.6,-0.5,2.1
590,Gary Clark,FR,2015,F,Cincinnati,AAC,0.543,0.524,13.4,18.9,...,5.5,13.5,16.9,289,2.2,2.4,4.7,4.6,4.1,8.7
491,Gary Clark,SO,2016,F,Cincinnati,AAC,0.578,0.545,12.7,19.6,...,5.4,12.4,17.3,367,2.9,2.5,5.4,4.7,4.3,9.0
273,Gary Clark,JR,2017,F,Cincinnati,AAC,0.585,0.554,12.3,19.5,...,4.8,11.2,18.8,411,3.4,2.5,5.9,5.8,4.0,9.9
11,Gary Clark,SR,2018,F,Cincinnati,AAC,0.612,0.571,12.1,22.3,...,5.0,8.7,20.4,462,4.3,3.5,7.8,7.9,6.4,14.4


In [8]:
#the model will consider this as two different Brandon Clarkes (even though it's the same player)
#Only the year at Gonzage will count as in the future I will merge by year of the combine and that is 2019 for Clarke
example2 = stats[stats["Player"] == "Brandon Clarke"].sort_values(by="Season")
example2 

,Player,Class,Season,Pos,School,Conf,TS%,eFG%,ORB%,DRB%,...,BLK%,TOV%,USG%,PProd,OWS,DWS,WS,OBPM,DBPM,BPM
3187,Brandon Clarke,FR,2016,F,San Jose State,MWC,0.627,0.637,9.7,16.6,...,5.6,13.5,17.0,270,1.8,1.1,2.9,3.3,2.1,5.4
1073,Brandon Clarke,SO,2017,F,San Jose State,MWC,0.595,0.593,10.7,21.8,...,8.5,10.8,25.3,504,3.1,1.7,4.8,5.2,2.5,7.7
3,Brandon Clarke,JR,2019,F,Gonzaga,WCC,0.699,0.693,13.9,19.6,...,11.3,11.3,23.9,603,5.5,3.2,8.8,9.8,6.5,16.3


**There are reasons I am ok with both of these downsides. 1) I find it very hard to believe that it is common for two players with the same name to have played at the same school in less than a decades time. There may be a handful of cases of this, if there are cases at all, but certainly not enought to heavily impact the data. Additionally, a player of that name would have needed to meet the criteria (GP and MP) in their 3rd NBA year to even make the final dataset, so that will whittle down the possible instances of this even further. 2) In the case of transfers, although ideally I would be able to account for every college year a player played, I don't think it's the end of the world if I only take the years from a player's final school before the draft (I take the player's final year in the weighted average, so I can match it up with the combine year). I already put more weight on more recent years in the weighted average, so I don't view putting a little extra weight on those years as something that will be a major issue. Additionally, it could even be argued that there is no reason to account for a player's stats at previous school at all, since they could have had a drastically different role or style at their old school that was not indicative of their true performance.**

In [9]:
#this function takes the stats dataframe, loops through each unique player in it and takes the weighted average of 
#their college stats so that each player has one row (with the exception of the instances I mentioned above)
#this function writes the final dataframe to a file
def combine_player_stats(df=stats):
    all_players = []
    for player in df.Player.unique():
        
        #test to see if it works for player at two schools
        #if player != "Brandon Clarke":
        #    continue
        
        #test to see if it works on first couple players
        #if player == "Ethan Happ":
        #    break
        
    
        #get the dataframe with just the rows with the desired player
        player_df = df[df.Player == player].sort_values(by="Season")
        #if there are multiple schools for that player, we further split into smaller dataframes by player and school
        #see markdown comment above for more information
        if len(player_df.School.unique()) > 1: 
            #loop through each school for player with the same name
            for school in player_df.School.unique():
                #make a df for each school
                player_school_df = player_df[player_df.School == school].sort_values(by="Season")
                #take the proper weighted average
                weighted_player_list = weighted_avg(player_school_df)
                #append to the final list of lists
                all_players.append(weighted_player_list)
                
    
        else: #if this condition is met, then the player only played for one team, no need to do anything else to player_df
            weighted_player_list = weighted_avg(player_df)
            all_players.append(weighted_player_list)
        
    
    #turn list of lists into a dataframe
    final_df = pd.DataFrame(all_players, columns = list(df.columns))
    #filename for the file to be downloaded
    filename = "combined_college_stats.csv"
    #THIS IF FOR WHEN I DO THE 2022 STATS
    filename2 = "combined_2022.csv"
    #write filename
    if not os.path.exists(filename2):
        final_df.to_csv(filename2, index=False)
        return(f"Combined player stats into one row per player via weighted average and downloaded to {filename2}")
    else:
        return(f"File {filename2} already exists")
        

In [10]:
#t1 = time.time()
#combine_player_stats()
#t2 = time.time()

In [11]:
#(t2-t1)/60 #takes only about 45 seconds

**The next part will be adding the NBA combine data.**

In [12]:
#read in college stats
college = pd.read_csv("combined_college_stats.csv")
#combine information gives more detailed position
college = college.drop(columns="Pos")
#change the type of season to int
college["Season"] = college["Season"].astype(int)
#inspect the data
college.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,BLK%,TOV%,USG%,PProd,OWS,DWS,WS,OBPM,DBPM,BPM
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,5.800,12.800,28.600,663.0,5.500,2.80,8.300,13.400,6.700,20.100
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,13.700,8.600,18.800,550.0,5.900,4.10,9.900,9.100,8.100,17.200
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,2.607,13.527,26.680,542.6,3.047,2.36,5.427,6.913,4.540,11.413
3,Brandon Clarke,SO,2017,San Jose State,MWC,0.606,0.608,10.367,20.067,15.100,...,7.533,11.700,22.533,426.0,2.667,1.50,4.167,4.567,2.367,6.933
4,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,11.300,11.300,23.900,603.0,5.500,3.20,8.800,9.800,6.500,16.300


**The file combine_measurables.csv comes from the combine_info.rmd R markdown code.**

In [13]:
#read in the measurable data
combine = pd.read_csv("combine_measurables.csv")
#subtract one from the year to get it to match up with the year of the combined player stats
combine["yearCombine"] = combine["yearCombine"] - 1
#filter out years after 2019 as these players won't have a 3rd year NBA BPM yet
combine = combine[combine["yearCombine"] <= 2019]
#drop lane agility and body fat% because there are missing values that I will not be able to find
combine = combine.drop(columns=["Lane Agiity", "Body Fat %"])
#rename yearCombine to season to match previous dataframe
combine = combine.rename(columns={"yearCombine": "Season"})
#inspect data
combine.tail()

,Season,Player,Position,Height,Weight,Wingspan,Standing Reach
605,2019,Coby White,PG,75.50,191.4,77.00,97.5
606,2019,Kris Wilkes,SF,78.25,208.8,82.75,103.0
607,2019,Grant Williams,PF,77.75,240.2,81.75,104.5
608,2019,Zion Williamson,PF,NaN,NaN,NaN,NaN
609,2019,Dylan Windler,SF,78.25,195.8,82.00,104.5


**For these 11 players that have some missing data, I am going to attempt to find the measurables that are missing and manually input the values. These may not be 100% accurate, but they are likely to be more accurate than doing some imputation. I found these numbers from various websites such as draftexpress.com, nbadraft.net, and craftednba.com**

In [14]:
#get index of the rows with missing data into a list
drop_indices = list(combine[combine.isnull().any(axis=1)].index)
#find rows where there are na values in an attempt to fill them
combine[combine.isnull().any(axis=1)]

,Season,Player,Position,Height,Weight,Wingspan,Standing Reach
200,2013,Alex Len,C,NaN,NaN,87.5,NaN
359,2016,Kris Dunn,PG,75.0,NaN,81.5,100.0
425,2017,Markelle Fultz,PG,NaN,NaN,NaN,NaN
534,2019,RJ Barrett,SF,NaN,NaN,NaN,NaN
555,2019,Darius Garland,PG,NaN,NaN,NaN,NaN
565,2019,De'Andre Hunter,NaN,NaN,NaN,NaN,NaN
572,2019,Romeo Langford,SG,76.5,NaN,83.0,103.0
580,2019,Ja Morant,PG,NaN,NaN,NaN,NaN
584,2019,Chuma Okeke,NaN,NaN,NaN,NaN,NaN
601,2019,Killian Tillie,NaN,NaN,NaN,NaN,NaN


In [15]:
#each list is one of the player's with missing data
alex_len = [2013, "Alex Len", "C", 83.5, 255.0, 87.5, 109.5]
kris_dunn = [2016, "Kris Dunn", "PG", 75.0, 205.0, 81.5, 100]
markelle_fultz = [2017, "Markelle Fultz", "PG", 75.0, 185, 81, 101]
rj_barrett = [2019, "RJ Barrett", "SF", 77.0, 208, 82, 102]
darius_garland = [2019, "Darius Garland", "PG", 73, 173, 77, 98]
deandre_hunter = [2019, "De'Andre Hunter", "SF", 79, 225, 86, 105]
romeo_langford = [2019, "Romeo Langford", "SG", 76.5, 210, 83.0, 103.0]
ja_morant = [2019, "Ja Morant", "PG", 74.5, 175, 78, 99]
chuma_okeke = [2019, "Chuma Okeke", "PF", 79, 235, 84, 103]
killian_tillie = [2019, "Killian Tillie", "PF", 80.75, 221.6, 80, 106]
zion_williamson = [2019, "Zion Williamson", "PF", 77.75, 272, 82.5, 103]
#make a list of lists of the players to be turned into a dataframe
missing_lofl = [alex_len, kris_dunn, markelle_fultz, rj_barrett, darius_garland, deandre_hunter, romeo_langford,
               ja_morant, chuma_okeke, killian_tillie, zion_williamson]
#create dataframe with filled in data for players that had missing data
missing = pd.DataFrame(missing_lofl, columns=combine.columns)
#missing #inspect the dataframe
#drop rows with the missing data in the combine dataset as they will be filled in the new dataframe
combine = combine.drop(drop_indices)
#Merge combine and missing dataframes
combine = combine.append(missing).sort_values(by="Season").reset_index(drop=True)
#check to make sure this combined dataset has 610 rows with no missing data
assert len(combine) == 610


In [16]:
#I'm going to look for duplicated players and keep only the ones that went to the NBA 
combined_df = college.merge(combine, on=["Season", "Player"])
combined_df[combined_df.duplicated(subset="Player", keep=False)]
#drop indices of the players I don't need
combined_df = combined_df.drop([65, 185, 252, 253, 315, 316, 390, 391]).reset_index(drop=True)
#inspect df
combined_df.head() 

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,DWS,WS,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,2.800,8.300,13.400,6.70,20.100,PF,77.75,272.0,82.50,103.0
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,4.100,9.900,9.100,8.10,17.200,PF,81.25,221.8,89.50,108.0
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,2.360,5.427,6.913,4.54,11.413,SG,75.50,211.6,82.00,103.0
3,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,3.200,8.800,9.800,6.50,16.300,SF,79.25,207.2,80.25,102.0
4,Frank Kaminsky,SR,2015,Wisconsin,Big Ten,0.606,0.570,7.527,21.327,14.560,...,2.227,7.227,8.853,4.80,13.653,C,83.75,231.2,83.00,109.5


**The next step is to add RSCI ranking data. The RSCI data is collected from Draft Express**

In [17]:
#this dictionary is mapping the year of the player to the number of years since they were a high school senior
#which is when their RSCI ranking is from
d = {"FR": 1, "SO": 2, "JR": 3, "SR": 4}
#create a new column with years since the RSCI ranking
combined_df["years_since"] = combined_df["Class"].map(d)
#create a column with the year of the RSCI ranking
combined_df["RSCI_Season"] = combined_df["Season"] - combined_df["years_since"]
#drop years since column since it isn't needed anymore
combined_df = combined_df.drop(columns="years_since")
#inspect df
combined_df.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,WS,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Season
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,8.300,13.400,6.70,20.100,PF,77.75,272.0,82.50,103.0,2018
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,9.900,9.100,8.10,17.200,PF,81.25,221.8,89.50,108.0,2011
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,5.427,6.913,4.54,11.413,SG,75.50,211.6,82.00,103.0,2013
3,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,8.800,9.800,6.50,16.300,SF,79.25,207.2,80.25,102.0,2016
4,Frank Kaminsky,SR,2015,Wisconsin,Big Ten,0.606,0.570,7.527,21.327,14.560,...,7.227,8.853,4.80,13.653,C,83.75,231.2,83.00,109.5,2011


In [18]:
#this function scrapes RSCI data from draft express between two years and writes it to a file called RSCI_rankings.csv
def scrape_rsci(year1, year2, big_df=pd.DataFrame()):
    for year in range(year1, year2+1):
        url = f"http://www.draftexpress.com/RSCI/{year}/"
        print("Visiting: " + url)
        page = requests.get(url)
        page.raise_for_status()
        df = pd.read_html(page.text)[0]
        df = df.rename(columns={df.columns[0]: "RSCI_Ranking"})
        df = df[["Player", "RSCI_Ranking"]]
        df["RSCI_Season"] = year
        big_df = big_df.append(df)
        time.sleep(1)
    
    filename = "RSCI_rankings.csv"
    #***FOR WHEN I'M GETTING THE 2022 DATA
    filename2 = "RSCI_rankings_2022.csv"
    if not os.path.exists(filename2):
        big_df.to_csv(filename2, index=False)
        return(f"Downloaded RSCI ranking data from {year1} to {year2} to a filename {filename2}")
    else:
        return(f"Filename {filename2} already exists")

In [19]:
#t1 = time.time()
#scrape_rsci(combined_df.RSCI_Season.min(), combined_df.RSCI_Season.max())
#t2 = time.time()

In [20]:
#(t2-t1) / 60 #only takes about 40 seconds

In [21]:
#read in rsci data
rsci = pd.read_csv("RSCI_rankings.csv")
#inspect df
rsci.head()

,Player,RSCI_Ranking,RSCI_Season
0,O.J. Mayo,1,2007
1,Kevin Love,2,2007
2,Eric Gordon,3,2007
3,Michael Beasley,4,2007
4,Derrick Rose,5,2007


In [22]:
#combined college stats with rsci ranking, want to left merge to keep every player since some players weren't ranked
combined_with_rsci = combined_df.merge(rsci, how="left", on=["Player", "RSCI_Season"])
#inspect
combined_with_rsci.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Season,RSCI_Ranking
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,13.400,6.70,20.100,PF,77.75,272.0,82.50,103.0,2018,5.0
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,9.100,8.10,17.200,PF,81.25,221.8,89.50,108.0,2011,1.0
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,6.913,4.54,11.413,SG,75.50,211.6,82.00,103.0,2013,36.0
3,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,9.800,6.50,16.300,SF,79.25,207.2,80.25,102.0,2016,NaN
4,Frank Kaminsky,SR,2015,Wisconsin,Big Ten,0.606,0.570,7.527,21.327,14.560,...,8.853,4.80,13.653,C,83.75,231.2,83.00,109.5,2011,NaN


In [23]:
combined_with_rsci["Season"].max()

2019

**There are a lot on NA values for RSCI_Rankings which I attribute to 2 things: 1) There are a good amount of players that are unranked by this metric 2) Transfers or redshirts that had to sit out a year will appear as Juniors (which I have attributed to 3 years since RSCI ranking in the RSCI_Season columm) for example even though it has really been 4 years since their RSCI ranking. To handle this I will first loop through these players and look to see if there is an RSCI ranking under this name. If there is, I will check to make sure that the year is one less than the RSCI_Season column. If this is the case, I will assume that the player transferred and had to sit out a year which is why the years aren't lined up. If this is not the case and the player was unranked, I will simply impute their RSCI_Ranking as the max ranking.**

**If there is a different player (Player B) with the same name as the desired player (Player A) in the year before the RSCI_Season column for the Player A, and player A truly is unranked, then there will be an error as Player A will be assigned Player B's RSCI ranking. Once again, I am ok with this potential error as it is unlikely that this happens enough to make a significant difference**

In [24]:
for i in range(len(combined_with_rsci)):
    val = combined_with_rsci.iloc[i]["RSCI_Ranking"] #this is the RSCI ranking for a player
    name = combined_with_rsci.iloc[i]["Player"] #this is the name of the player
    if np.isnan(val): #if this value is null this condition will be true
        #if this is true check to see if they sat out a year by adjusted the year to be one year before
        adjusted_year = combined_with_rsci.iloc[i]["RSCI_Season"] - 1
        #check to see if there an instance of the player and their adjusted year in the rsci dataframe
        name_df = rsci[(rsci["Player"] == name) & (rsci["RSCI_Season"] == adjusted_year)]
        #if there is an instance the true rsci ranking will be the rsci ranking of the adjusted year
        if len(name_df) == 1:
            true_rsci = int(name_df["RSCI_Ranking"])
        #if there are no instances of this, then the player most likely truly was unranked 
        elif len(name_df) == 0:
            #if the player was unranked take their year
            year = combined_with_rsci.iloc[i]["RSCI_Season"]
            #assign their rsci ranking to 1 more than the max from that year
            true_rsci = rsci[rsci["RSCI_Season"] == year]["RSCI_Ranking"].max() + 1
        #fill na values with the true rsci ranking
        combined_with_rsci.iat[i, -1] = true_rsci
    else: #if value isn't null no need to mess with it
        continue

#ensure that there are no null values and that all were filled
assert len(combined_with_rsci[combined_with_rsci["RSCI_Ranking"].isnull()]) == 0
#don't need RSCI_Season column anymore
combined_with_rsci = combined_with_rsci.drop(columns="RSCI_Season")
#inspect df
combined_with_rsci.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,WS,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,8.300,13.400,6.70,20.100,PF,77.75,272.0,82.50,103.0,5.0
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,9.900,9.100,8.10,17.200,PF,81.25,221.8,89.50,108.0,1.0
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,5.427,6.913,4.54,11.413,SG,75.50,211.6,82.00,103.0,36.0
3,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,8.800,9.800,6.50,16.300,SF,79.25,207.2,80.25,102.0,256.0
4,Frank Kaminsky,SR,2015,Wisconsin,Big Ten,0.606,0.570,7.527,21.327,14.560,...,7.227,8.853,4.80,13.653,C,83.75,231.2,83.00,109.5,187.0


**The next step is to add ages by getting the players birthdays. There is a kaggle dataset that contains players birthdays and start years that I am going to download to use to get the draft age. The one downside is that it is not updated through the present so I will have to manually fill birthdays of players drafted in 2018 and 2019**

**The url for this kaggle data set is https://www.kaggle.com/datasets/drgilermo/nba-players-stats and the correct one to download is player_data.csv. Additionally, I will be renaming this file player_birthdays.csv when I download it from Kaggle.**

In [25]:
#read in the file
birthdays = pd.read_csv("player_birthdays.csv")
#subtract 1 from the year start so it matches up with the year of the combine and draft
birthdays["year_start"] = birthdays["year_start"] - 1
#filter only the years of interest
birthdays = birthdays[birthdays["year_start"] >= 2011]
#drop columns that we don't need
birthdays = birthdays.drop(columns=["year_end", "position", "height", "weight", "college"])
#rename columns
birthdays = birthdays.rename(columns={"name": "Player", "year_start": "Season", "birth_date": "Birthday"})
#inspect df
birthdays.head()

,Player,Season,Birthday
9,Alex Abrines,2016,"August 1, 1993"
14,Quincy Acy,2012,"October 6, 1990"
19,Jordan Adams,2014,"July 8, 1994"
21,Steven Adams,2013,"July 20, 1993"
23,Bam Adebayo,2017,"July 18, 1997"


**I am going to map each season to a draft date.**

In [26]:
#dictionary containing the date of each year's draft
draft_dict = {2011: "June 23, 2011", 2012: "June 28, 2012", 2013: "June 27, 2013", 2014: "June 26, 2014",
              2015: "June 25, 2015", 2016: "June 23, 2016", 2017: "June 22, 2017", 2018: "June 21, 2018",
              2019: "June 20, 2019"}
#add columns with draft date
combined_with_rsci["Draft_Date"] = combined_with_rsci["Season"].map(draft_dict)
#inspect df
combined_with_rsci.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking,Draft_Date
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,13.400,6.70,20.100,PF,77.75,272.0,82.50,103.0,5.0,"June 20, 2019"
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,9.100,8.10,17.200,PF,81.25,221.8,89.50,108.0,1.0,"June 28, 2012"
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,6.913,4.54,11.413,SG,75.50,211.6,82.00,103.0,36.0,"June 22, 2017"
3,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,9.800,6.50,16.300,SF,79.25,207.2,80.25,102.0,256.0,"June 20, 2019"
4,Frank Kaminsky,SR,2015,Wisconsin,Big Ten,0.606,0.570,7.527,21.327,14.560,...,8.853,4.80,13.653,C,83.75,231.2,83.00,109.5,187.0,"June 25, 2015"


In [27]:
#merge the birthdays dataframe with the other stats
total = combined_with_rsci.merge(birthdays, how="left", on=["Player", "Season"])
#inspect (Notice 2018 and 2019 are NA)
total.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking,Draft_Date,Birthday
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,6.70,20.100,PF,77.75,272.0,82.50,103.0,5.0,"June 20, 2019",NaN
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,8.10,17.200,PF,81.25,221.8,89.50,108.0,1.0,"June 28, 2012","March 11, 1993"
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,4.54,11.413,SG,75.50,211.6,82.00,103.0,36.0,"June 22, 2017","November 15, 1994"
3,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,6.50,16.300,SF,79.25,207.2,80.25,102.0,256.0,"June 20, 2019",NaN
4,Frank Kaminsky,SR,2015,Wisconsin,Big Ten,0.606,0.570,7.527,21.327,14.560,...,4.80,13.653,C,83.75,231.2,83.00,109.5,187.0,"June 25, 2015","April 4, 1993"


**There are a ton of players with missing birthdays so I am going to look them up and fill them. If the player does not have 3rd year NBA stats I will not fill their row (will drop them at the end). For example Xavier Thames did not play a 3rd nba year so I'm not going to bother adding his birthday**

In [28]:
#add birthdays
total.iat[0, -1] = "July 6, 2000" #Zion Williamson
total.iat[3, -1] = "September, 1996" #Brandon Clarke
total.iat[13, -1] = "March 3, 1996" #Cameron Johnson
total.iat[18, -1] = "November 30, 1998" #Grant Williams
total.iat[26, -1] = "March 22, 1994" #Taurean Prince
total.iat[28, -1] = "September 14, 1995" #Jevon Carter
total.iat[35, -1] = "September 15, 1999" #Jaren Jackson Jr.
total.iat[39, -1] = "February 20, 1999" #Jarret Culver
total.iat[44, -1] = "July 8, 1997" #Ty Jerome
total.iat[54, -1] = "January 23, 1996" # Keita Bates-Diop
total.iat[58, -1] = "September 19, 1998" # Trae Young
total.iat[61, -1] = "May 12, 1998" # Mohammed Bamba
total.iat[66, -1] = "December 2, 1997" #De'Andre Hunter
total.iat[80, -1] = "March 28, 1991" #Jordan McRae
total.iat[83, -1] = "January 11, 1993" #Chris Boucher
total.iat[87, -1] = "March 21, 1998" #Miles Bridges
total.iat[88, -1] = "January 26, 1989" #Marshon Brooks
total.iat[94, -1] = "September 22, 1996" #Dylan Windler
total.iat[95, -1] = "August 31, 1996" #Jalen Brunson
total.iat[101, -1] = "December 2, 1994" #Kenrich Williams
total.iat[104, -1] = "May 23, 2000" #Jaxson Hayes
total.iat[107, -1] = "August 10, 1999" #Ja Morant
total.iat[109, -1] = "October 8, 1995" #Grayson Allen
total.iat[157, -1] = "June 21, 1990" #JaMychal Green
total.iat[160, -1] = "April 26, 1997" #Moritz Wagner
total.iat[167, -1] = "January 31, 1997" #Donte DiVincenzo
total.iat[169, -1] = "July 12, 1998" #Shai Gilgeous-Alexander
total.iat[175, -1] = "September 2, 1998" #Nickeil Alexander-Walker
total.iat[177, -1] = "January 8, 1999" #Ignas Brazdeikis
total.iat[179, -1] = "October 28, 1993" #Treveon Graham
total.iat[180, -1] = "January 20, 2000" #Tyler Herro
total.iat[188, -1] = "March 23, 1993" #Quinn Cook
total.iat[191, -1] = "March 13, 1997" #Landry Shamet
total.iat[198, -1] = "September 28, 1992" #Khem Birch
total.iat[214, -1] = "October 28, 1994" #Andrew Harrison
total.iat[222, -1] = "October 1, 1998" #Daniel Gafford
total.iat[225, -1] = "July 4, 1994" #DeAndre Bembry
total.iat[239, -1] = "September 1, 1998" #Josh Okogie
total.iat[245, -1] = "February 16, 2000" #Coby White
total.iat[251, -1] = "February 3, 1998" #Isaiah Roby
total.iat[253, -1] = "April 3, 1991" #Hollis Thompson
total.iat[257, -1] = "August 27, 1998" #Kevin Huerter
total.iat[261, -1] = "May 16, 1997" #Terence Davis
total.iat[265, -1] = "October 18, 1996" #Terance Mann
total.iat[269, -1] = "September 28, 1995" #Cody Martin
total.iat[281, -1] = "January 31, 1998" #Jalen McDaniels
total.iat[282, -1] = "November 4, 1996" #Eric Paschall
total.iat[295, -1] = "June 19, 1999" #Jordan Poole
total.iat[315, -1] = "January 18, 1999" #Gary Trent
total.iat[318, -1] = "January 4, 1999" #Collin Sexton
total.iat[335, -1] = "October 11, 1999" #Keldon Johnson
total.iat[340, -1] = "June 10, 1997" #Sviatoslav Mykhailiuk
total.iat[345, -1] = "October 25, 1999" #Romeo Langford
total.iat[350, -1] = "May 4, 1988" #Kyle Singler
total.iat[369, -1] = "September 30, 1996" #Aaron Holiday
total.iat[382, -1] = "April 17, 1999" #Nicolas Claxton
total.iat[392, -1] = "November 1, 1992" #Semaj Christon
total.iat[395, -1] = "December 14, 1998" #Lonnie Walker
total.iat[396, -1] = "May 4, 1998" #Frank Jackson
total.iat[397, -1] = "August 11, 1999" #Kevin Knox
total.iat[398, -1] = "February 11, 2000" #Nassir Little
total.iat[405, -1] = "July 9, 1999" #Jaylen Nowell
total.iat[419, -1] = "November 25, 2000" #Talen Horton-Tucker
total.iat[425, -1] = "July 28, 1999" #Troy Brown
total.iat[432, -1] = "April 19, 1999" #Luguentz Dort
total.iat[434, -1] = "July 31, 1998" #Hamidou Diallo
total.iat[437, -1] = "August 26, 1999" #Naz Reid
#drop na rows
total = total.dropna().reset_index(drop=True)
#make sure there are no na rows left
assert len(total[total.Birthday.isnull()]) == 0
#inspect df
total.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking,Draft_Date,Birthday
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,6.70,20.100,PF,77.75,272.0,82.50,103.0,5.0,"June 20, 2019","July 6, 2000"
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,8.10,17.200,PF,81.25,221.8,89.50,108.0,1.0,"June 28, 2012","March 11, 1993"
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,4.54,11.413,SG,75.50,211.6,82.00,103.0,36.0,"June 22, 2017","November 15, 1994"
3,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,6.50,16.300,SF,79.25,207.2,80.25,102.0,256.0,"June 20, 2019","September, 1996"
4,Frank Kaminsky,SR,2015,Wisconsin,Big Ten,0.606,0.570,7.527,21.327,14.560,...,4.80,13.653,C,83.75,231.2,83.00,109.5,187.0,"June 25, 2015","April 4, 1993"


**Now since I have birth date and draft date, I am able to subtract them to find the players age on draft day**

In [29]:
total["Draft_Date"] = pd.to_datetime(total["Draft_Date"].str.replace(",", ""))
total["Birthday"] = pd.to_datetime(total["Birthday"].str.replace(",", ""))
#create a column with the draft age
total["Draft_Age"] = total["Draft_Date"] - total["Birthday"]
#this function changes the result from days to years...will be applied to the "Draft_Age" column
def convert_to_years(date):
    return round((date.total_seconds() / 60 / 60 / 24 / 365.25), 2)
#apply function to the column to get years
total["Draft_Age"] = total["Draft_Age"].apply(convert_to_years)
#drop the birth date and draft date columns since I now have the "Draft_Age" column
total = total.drop(columns=["Birthday", "Draft_Date"])
#inspect the df
total.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking,Draft_Age
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,13.400,6.70,20.100,PF,77.75,272.0,82.50,103.0,5.0,18.95
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,9.100,8.10,17.200,PF,81.25,221.8,89.50,108.0,1.0,19.30
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,6.913,4.54,11.413,SG,75.50,211.6,82.00,103.0,36.0,22.60
3,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,9.800,6.50,16.300,SF,79.25,207.2,80.25,102.0,256.0,22.80
4,Frank Kaminsky,SR,2015,Wisconsin,Big Ten,0.606,0.570,7.527,21.327,14.560,...,8.853,4.80,13.653,C,83.75,231.2,83.00,109.5,187.0,22.22


In [30]:
#just fun to look at the youngest and oldest prospects
total.sort_values(by="Draft_Age")

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking,Draft_Age
300,Talen Horton-Tucker,FR,2019,Iowa State,Big 12,0.488,0.470,2.900,16.700,10.200,...,1.700,2.500,4.200,SG,74.50,235.4,85.25,103.0,67.0,18.57
110,Devin Booker,FR,2015,Kentucky,SEC,0.600,0.571,2.600,7.800,5.400,...,6.100,3.300,9.400,SG,76.50,205.8,80.25,102.5,20.0,18.65
318,Ike Anigbogu,FR,2017,UCLA,Pac-12,0.564,0.564,13.200,19.900,16.900,...,-1.500,2.400,0.900,C,80.50,252.2,90.25,110.5,53.0,18.67
204,Michael Kidd-Gilchrist,FR,2012,Kentucky,SEC,0.570,0.511,10.300,17.300,14.000,...,4.100,3.600,7.700,SF,77.75,232.8,84.00,104.5,3.0,18.75
33,Jaren Jackson,FR,2018,Michigan State,Big Ten,0.647,0.595,8.800,19.700,15.000,...,4.400,7.300,11.700,PF,81.75,236.0,89.25,110.0,9.0,18.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Kadeem Allen,SR,2017,Arizona,Pac-12,0.563,0.516,1.400,13.333,7.800,...,2.167,5.033,7.200,PG,73.00,192.4,81.25,99.5,261.0,24.43
69,Chris Boucher,SR,2017,Oregon,Pac-12,0.604,0.596,10.133,20.200,15.433,...,4.767,4.000,8.800,PF,80.00,182.2,88.00,110.5,261.0,24.44
227,Jamel Artis,SR,2017,Pittsburgh,ACC,0.586,0.548,5.213,13.187,9.240,...,5.767,0.353,6.120,SG,77.50,213.2,82.00,105.0,192.0,24.44
34,Sean Kilpatrick,SR,2014,Cincinnati,AAC,0.557,0.510,4.100,12.220,8.007,...,7.427,2.620,9.993,SG,74.50,218.7,79.00,99.0,151.0,24.47


**The "total" dataframe has all of the stats/measurables/rankings that I want so I am going to download this dataframe to a csv file named all_features.csv**

In [31]:
if not os.path.exists("all_features.csv"):
    total.to_csv("all_features.csv", index=False)
    print("Downloaded file with college player stats, measurables, and rsci rankings to file all_features.csv")
else:
    print("File called all_features.csv already downloaded")

File called all_features.csv already downloaded


**The next step is to merge all the feature variables containing statistics and information about a player with the target variable of 3rd year BPM. How I define 3rd year BPM is the 3rd year after the leaving college. So, if a player played in Europe for 5 years after college and then played 3 years in the NBA, they will not be included in this dataset. I'm strictly worried about the question of "If I draft this player, how will they be at the end of their rookie contract".**

**I get the csv file that I use here from the target_data_r.rmd file because of the ability to use the nbastatr package to easily acquire OBPM and DBPM from basketball reference using this package. You can see more details in that file; however, the only really important thing to know is that I filtered out seasons where players did not play 700 minutes or more because I wanted to have a large enough sample size to get a reliable measure of OBPM and DBPM.**

In [32]:
#read in the data
target = pd.read_csv("professional_data.csv")
feature = pd.read_csv("all_features.csv")
#drop tony mitchell as he is causing errors due to showing up twice and doesn't have a 3rd year BPM anyways
feature = feature.drop([117, 118])
#rename column
target = target.rename(columns={"Year": "Season"})
#inspect target
target.head()

,Season,Player,Position,OBPM,DBPM
0,2011,Arron Afflalo,SG,1.4,-1.2
1,2011,LaMarcus Aldridge,PF,3.4,-0.4
2,2011,Ray Allen,SG,1.5,0.7
3,2011,Tony Allen,SG,-0.6,2.3
4,2011,Al-Farouq Aminu,SF,-2.9,0.7


**The function below takes the feature and target datasets and combines them. In other words, this function combines a players college stats, measurables, rankigns, etc. and the player's 3rd NBA year OBPM, DBPM, and BPM. The end result is a file with 172 rows with no missing data. This function saves this combined dataframe to a file called modeling.csv which contains all the necessary information needed to model.**

In [33]:
def get_3rd_year(feature_df = feature, target_df = target):
    #initialize lists to append 3rd year OBPM and DBPM to
    obpm = []
    dbpm = []
    #loop through players in feature df
    for player in feature_df.Player.unique():
        #the season in the feature df is the year of the combine, draft, and start of their first NBA season
        #therefore their 3rd season would be 2 years after their first season
        year3 = int(feature_df[feature_df.Player == player]["Season"]) + 2
        #if the player does not have a 3rd year OBPM and DBPM we will append na
        if len(target_df[(target_df.Player == player) & (target_df.Season == year3)]) == 0:
            player_obpm = np.nan
            player_dbpm = np.nan
        #else get their 3rd year obpm and dbpm
        elif len(target_df[(target_df.Player == player) & (target_df.Season == year3)]) == 1:
            player_obpm = float(target_df[(target_df.Player == player) & (target_df.Season == year3)]["OBPM"])
            player_dbpm = float(target_df[(target_df.Player == player) & (target_df.Season == year3)]["DBPM"])
        
        #append to the list
        obpm.append(player_obpm)
        dbpm.append(player_dbpm)
    
    #convert lists to numpy arrays that represent columns that will be added to the dataframe
    season3_OBPM = np.array(obpm)
    season3_DBPM = np.array(dbpm)
    #initialize the final dataframe
    final = feature_df
    #add columns to final
    final["Season3 OBPM"] = season3_OBPM
    final["Season3 DBPM"] = season3_DBPM
    final["Season3 BPM"] = final["Season3 OBPM"] + final["Season3 DBPM"]
    #drop na 
    final = final.dropna().reset_index(drop=True)
    #order columns how I want them (not including some columns in this list to drop them---Win Shares and Conference)
    column_order = ["Player", "Position", "Season", "Draft_Age", "Class", "School", "Height", "Weight", "Wingspan", 
                "Standing Reach", "RSCI_Ranking", "TS%", "eFG%", "ORB%", "DRB%", "TRB%", "AST%", "STL%", 
                "BLK%", "TOV%", "USG%", "PProd", "OBPM", "DBPM", "BPM", "Season3 OBPM", "Season3 DBPM", 
                "Season3 BPM"]
    final = final[column_order]
    #rename column names to get them to be more uniform
    final = final.rename(columns = {"Season": "Draft Year", "RSCI_Ranking": "RSCI Ranking", "Draft_Age": "Draft Age",
                                   "OBPM": "College OBPM", "DBPM": "College DBPM", "BPM": "College BPM"})
    #save to file modeling.csv
    filename = "modeling.csv"
    if not os.path.exists(filename):
        final.to_csv(filename, index=False)
        return(f"Saved final dataset that has everything needed to model to {filename}")
    else:
        return(f"File {filename} already exists")
        
        

In [34]:
#get_3rd_year()

**The next part will be to get the dataset for 2022 so that I can make predictions on it.**

In [35]:
stats_2022 = pd.read_csv("college_advanced.csv") # read in data
#keep only selected columns
stats_2022 = stats_2022[["Player", "Class", "Season", "Pos", "School", "Conf", "TS%", "eFG%", "ORB%", "DRB%", "TRB%", "AST%", 
              "STL%", "BLK%", "TOV%", "USG%", "PProd", "OWS", "DWS", "WS", "OBPM", "DBPM", "BPM"]]
#split the hyphenated season into start year and end year
stats_2022[["Start_Year", "End_Year"]] = stats_2022["Season"].str.split('-', expand=True)
#edit the season column so that it just keeps the end year (year of the NCAA tournament and the NBA combine)
stats_2022["Season"] = (stats_2022["Start_Year"].astype(int) + 1).astype(str)
#drop excess columns
stats_2022 = stats_2022.drop(columns=["Start_Year", "End_Year"])
stats_2022.head(10)

,Player,Class,Season,Pos,School,Conf,TS%,eFG%,ORB%,DRB%,...,BLK%,TOV%,USG%,PProd,OWS,DWS,WS,OBPM,DBPM,BPM
0,Zion Williamson,FR,2019,F,Duke,ACC,0.702,0.708,12.7,18.0,...,5.8,12.8,28.6,663,5.5,2.8,8.3,13.4,6.7,20.1
1,Anthony Davis,FR,2012,F,Kentucky,SEC,0.654,0.628,11.6,25.6,...,13.7,8.6,18.8,550,5.9,4.1,9.9,9.1,8.1,17.2
2,Sindarius Thornwell,SR,2017,G,South Carolina,SEC,0.590,0.510,8.0,16.3,...,3.4,11.9,29.5,634,4.4,2.8,7.2,11.0,6.2,17.1
3,Brandon Clarke,JR,2019,F,Gonzaga,WCC,0.699,0.693,13.9,19.6,...,11.3,11.3,23.9,603,5.5,3.2,8.8,9.8,6.5,16.3
4,Frank Kaminsky,SR,2015,C,Wisconsin,Big Ten,0.628,0.590,6.0,25.7,...,4.5,9.8,28.6,691,6.9,2.9,9.8,11.2,5.0,16.2
5,Keegan Murray,SO,2022,F,Iowa,Big Ten,0.638,0.614,9.8,20.2,...,6.4,5.7,29.7,724,6.6,2.0,8.7,13.0,2.7,15.7
6,Denzel Valentine,SR,2016,G,Michigan State,Big Ten,0.608,0.579,3.0,21.2,...,0.7,14.8,28.5,622,5.0,2.2,7.2,11.3,4.3,15.6
7,Delon Wright,SR,2015,G,Utah,Pac-12,0.619,0.549,4.0,13.7,...,3.2,14.2,22.8,558,5.3,3.1,8.4,8.7,6.9,15.5
8,Victor Oladipo,JR,2013,G,Indiana,Big Ten,0.671,0.648,11.8,15.1,...,2.8,18.5,22.2,478,4.2,2.6,6.7,8.9,6.2,15.1
9,Chet Holmgren,FR,2022,C,Gonzaga,WCC,0.691,0.680,8.0,28.7,...,12.6,15.7,21.6,424,3.3,3.1,6.3,7.5,7.5,15.0


In [36]:
#combine_player_stats(stats_2022)

In [37]:
#read in college_2022 stats
college_2022 = pd.read_csv("combined_2022.csv")
#combine information gives more detailed position
college_2022 = college_2022.drop(columns="Pos")
#change the type of season to int
college_2022["Season"] = college_2022["Season"].astype(int)
#inspect the data
college_2022.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,BLK%,TOV%,USG%,PProd,OWS,DWS,WS,OBPM,DBPM,BPM
0,Zion Williamson,FR,2019,Duke,ACC,0.702,0.708,12.700,18.000,15.500,...,5.800,12.800,28.600,663.0,5.500,2.80,8.300,13.400,6.700,20.100
1,Anthony Davis,FR,2012,Kentucky,SEC,0.654,0.628,11.600,25.600,19.000,...,13.700,8.600,18.800,550.0,5.900,4.10,9.900,9.100,8.100,17.200
2,Sindarius Thornwell,SR,2017,South Carolina,SEC,0.543,0.472,6.407,14.327,10.227,...,2.607,13.527,26.680,542.6,3.047,2.36,5.427,6.913,4.540,11.413
3,Brandon Clarke,SO,2017,San Jose State,MWC,0.606,0.608,10.367,20.067,15.100,...,7.533,11.700,22.533,426.0,2.667,1.50,4.167,4.567,2.367,6.933
4,Brandon Clarke,JR,2019,Gonzaga,WCC,0.699,0.693,13.900,19.600,17.100,...,11.300,11.300,23.900,603.0,5.500,3.20,8.800,9.800,6.500,16.300


In [38]:
combine = pd.read_csv("combine_measurables_2022.csv")
#subtract one from the year to get it to match up with the year of the combined player stats
combine["yearCombine"] = combine["yearCombine"] - 1
#drop lane agility and body fat% because there are missing values that I will not be able to find
combine = combine.drop(columns=["Lane Agiity", "Body Fat %"])
#rename yearCombine to season to match previous dataframe
combine = combine.rename(columns={"yearCombine": "Season"})
#inspect data
combine.tail()

,Season,Player,Position,Height,Weight,Wingspan,Standing Reach
78,2022,Bryson Williams,PF,NaN,NaN,NaN,NaN
79,2022,Mark Williams,C,84.0,242.4,90.50,117.0
80,2022,Jalen Williams,SG,76.5,209.2,86.25,105.5
81,2022,Jalen Wilson,SF,NaN,NaN,NaN,NaN
82,2022,Fanbo Zeng,SF,NaN,NaN,NaN,NaN


**Manually going to attempt to fill missing height, weight, wingspan, reach the same way as before**

In [39]:
drop_indices = list(combine[combine.isnull().any(axis=1)].index)
#combined_df[combined_df.isnull().any(axis=1)]
#drop_indices

In [40]:
#each list is one of the player's with missing data
keegan_murray = [2022, "Keegan Murray", "PF", 80, 225.0, 83, 104]
chet_holmgren = [2022, "Chet Holmgren", "PF", 84, 195, 90, 115]
jabari_smith = [2022, "Jabari Smith", "PF", 82, 220, 85, 110]
bryson_williams = [2022, "Bryson Williams", "PF", 80, 235, 86.25, 107]
darius_days = [2022, "Darius Days", "SF", 77.75, 245, 84.5, 107.5]
aj_griffin = [2022, "AJ Griffin", "SF", 78, 220, 84, 105.5]
jeremy_sochan = [2022, "Jeremy Sochan", "PF", 81, 230, 84, 108.5]
jalen_wilson = [2022, "Jalen Wilson", "SF", 77.75, 225.2, 80, 102]
paolo_banchero = [2022, "Paolo Banchero", "PF", 82, 250, 84.5, 108.5]
jaden_ivey = [2022, "Jaden Ivey", "PG", 76, 195, 81, 101.5]
jalen_duren = [2022, "Jalen Duren", "C", 81, 242.4, 89, 115]
tyrese_martin = [2022, "Tyrese Martin", "SG", 77.25, 215, 80, 103.5]
jared_rhoden = [2022, "Jared Rhoden", "SG", 78, 210, 82.5, 104]
#make a list of lists of the players to be turned into a dataframe
missing_lofl = [keegan_murray, chet_holmgren, jabari_smith, bryson_williams, darius_days, aj_griffin, jeremy_sochan,
               jalen_wilson, paolo_banchero, jaden_ivey, jalen_duren, tyrese_martin, jared_rhoden]
#create dataframe with filled in data for players that had missing data
missing = pd.DataFrame(missing_lofl, columns=combine.columns)
#missing #inspect the dataframe
#drop rows with the missing data in the combine dataset as they will be filled in the new dataframe
combine = combine.drop(drop_indices)
#Merge combine and missing dataframes
combine = combine.append(missing).sort_values(by="Season").reset_index(drop=True)
#check to make sure there are no null values
assert len(combined_df[combined_df.isnull().any(axis=1)]) == 0
combine.head()

,Season,Player,Position,Height,Weight,Wingspan,Standing Reach
0,2022,Ochai Agbaji,SG,76.50,216.8,82.25,104.0
1,2022,Jabari Walker,PF,78.75,213.8,82.75,105.0
2,2022,Drew Timme,C,80.25,231.6,86.00,107.5
3,2022,Dalen Terry,SG,78.00,195.2,84.75,106.0
4,2022,Julian Strawther,SF,77.75,207.6,81.50,104.5


In [41]:
#Combine dataframes 
combined_df = college_2022.merge(combine, on=["Season", "Player"])
#drop one duplicate 
combined_df = combined_df.drop(28).reset_index(drop=True)
#inspect df
combined_df.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,DWS,WS,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach
0,Keegan Murray,SO,2022,Iowa,Big Ten,0.621,0.594,9.800,19.933,14.867,...,1.700,6.700,10.3,2.933,13.233,PF,80.00,225.0,83.00,104.0
1,Chet Holmgren,FR,2022,Gonzaga,WCC,0.691,0.680,8.000,28.700,19.600,...,3.100,6.300,7.5,7.500,15.000,PF,84.00,195.0,90.00,115.0
2,Tari Eason,SO,2022,Louisiana State,SEC,0.615,0.559,10.500,19.800,15.200,...,2.500,5.900,9.0,5.700,14.700,PF,78.75,217.4,86.00,107.5
3,Walker Kessler,SO,2022,Auburn,SEC,0.627,0.627,10.900,22.200,16.600,...,2.700,5.500,6.2,8.000,14.100,C,84.25,256.0,88.25,113.0
4,Mark Williams,SO,2022,Duke,ACC,0.700,0.702,13.633,20.800,17.400,...,1.533,4.833,7.2,4.433,11.567,C,84.00,242.4,90.50,117.0


**Add RSCI data for 2022 in the same way as before**

In [42]:
#scrape_rsci(2016, 2021)

In [43]:
#this dictionary is mapping the year of the player to the number of years since they were a high school senior
#which is when their RSCI ranking is from
d = {"FR": 1, "SO": 2, "JR": 3, "SR": 4}
#create a new column with years since the RSCI ranking
combined_df["years_since"] = combined_df["Class"].map(d)
#create a column with the year of the RSCI ranking
combined_df["RSCI_Season"] = combined_df["Season"] - combined_df["years_since"]
#drop years since column since it isn't needed anymore
combined_df = combined_df.drop(columns="years_since")
#inspect df
combined_df.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,WS,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Season
0,Keegan Murray,SO,2022,Iowa,Big Ten,0.621,0.594,9.800,19.933,14.867,...,6.700,10.3,2.933,13.233,PF,80.00,225.0,83.00,104.0,2020
1,Chet Holmgren,FR,2022,Gonzaga,WCC,0.691,0.680,8.000,28.700,19.600,...,6.300,7.5,7.500,15.000,PF,84.00,195.0,90.00,115.0,2021
2,Tari Eason,SO,2022,Louisiana State,SEC,0.615,0.559,10.500,19.800,15.200,...,5.900,9.0,5.700,14.700,PF,78.75,217.4,86.00,107.5,2020
3,Walker Kessler,SO,2022,Auburn,SEC,0.627,0.627,10.900,22.200,16.600,...,5.500,6.2,8.000,14.100,C,84.25,256.0,88.25,113.0,2020
4,Mark Williams,SO,2022,Duke,ACC,0.700,0.702,13.633,20.800,17.400,...,4.833,7.2,4.433,11.567,C,84.00,242.4,90.50,117.0,2020


In [44]:
rsci = pd.read_csv("RSCI_rankings_2022.csv")
rsci

,Player,RSCI_Ranking,RSCI_Season
0,Josh Jackson,1,2016
1,Harry Giles,2,2016
2,Jayson Tatum,3,2016
3,Markelle Fultz,4,2016
4,Lonzo Ball,5,2016
...,...,...,...
2075,Quentin Diboundje,252,2021
2076,Jalen Lake,253,2021
2077,Scottie Ebube,254,2021
2078,Randy Brady,255,2021


In [45]:
#combined college stats with rsci ranking, want to left merge to keep every player since some players weren't ranked
combined_with_rsci = combined_df.merge(rsci, how="left", on=["Player", "RSCI_Season"])
#inspect
combined_with_rsci.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Season,RSCI_Ranking
0,Keegan Murray,SO,2022,Iowa,Big Ten,0.621,0.594,9.800,19.933,14.867,...,10.3,2.933,13.233,PF,80.00,225.0,83.00,104.0,2020,NaN
1,Chet Holmgren,FR,2022,Gonzaga,WCC,0.691,0.680,8.000,28.700,19.600,...,7.5,7.500,15.000,PF,84.00,195.0,90.00,115.0,2021,1.0
2,Tari Eason,SO,2022,Louisiana State,SEC,0.615,0.559,10.500,19.800,15.200,...,9.0,5.700,14.700,PF,78.75,217.4,86.00,107.5,2020,113.0
3,Walker Kessler,SO,2022,Auburn,SEC,0.627,0.627,10.900,22.200,16.600,...,6.2,8.000,14.100,C,84.25,256.0,88.25,113.0,2020,20.0
4,Mark Williams,SO,2022,Duke,ACC,0.700,0.702,13.633,20.800,17.400,...,7.2,4.433,11.567,C,84.00,242.4,90.50,117.0,2020,29.0


In [46]:
for i in range(len(combined_with_rsci)):
    val = combined_with_rsci.iloc[i]["RSCI_Ranking"] #this is the RSCI ranking for a player
    name = combined_with_rsci.iloc[i]["Player"] #this is the name of the player
    if np.isnan(val): #if this value is null this condition will be true
        #if this is true check to see if they sat out a year by adjusted the year to be one year before
        adjusted_year = combined_with_rsci.iloc[i]["RSCI_Season"] - 1
        #check to see if there an instance of the player and their adjusted year in the rsci dataframe
        name_df = rsci[(rsci["Player"] == name) & (rsci["RSCI_Season"] == adjusted_year)]
        #if there is an instance the true rsci ranking will be the rsci ranking of the adjusted year
        if len(name_df) == 1:
            true_rsci = int(name_df["RSCI_Ranking"])
        #if there are no instances of this, then the player most likely truly was unranked 
        elif len(name_df) == 0:
            #if the player was unranked take their year
            year = combined_with_rsci.iloc[i]["RSCI_Season"]
            #assign their rsci ranking to 1 more than the max from that year
            true_rsci = rsci[rsci["RSCI_Season"] == year]["RSCI_Ranking"].max() + 1
        #fill na values with the true rsci ranking
        combined_with_rsci.iat[i, -1] = true_rsci
    else: #if value isn't null no need to mess with it
        continue

#ensure that there are no null values and that all were filled
assert len(combined_with_rsci[combined_with_rsci["RSCI_Ranking"].isnull()]) == 0
#don't need RSCI_Season column anymore
combined_with_rsci = combined_with_rsci.drop(columns="RSCI_Season")
#inspect df
combined_with_rsci.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,WS,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking
0,Keegan Murray,SO,2022,Iowa,Big Ten,0.621,0.594,9.800,19.933,14.867,...,6.700,10.3,2.933,13.233,PF,80.00,225.0,83.00,104.0,518.0
1,Chet Holmgren,FR,2022,Gonzaga,WCC,0.691,0.680,8.000,28.700,19.600,...,6.300,7.5,7.500,15.000,PF,84.00,195.0,90.00,115.0,1.0
2,Tari Eason,SO,2022,Louisiana State,SEC,0.615,0.559,10.500,19.800,15.200,...,5.900,9.0,5.700,14.700,PF,78.75,217.4,86.00,107.5,113.0
3,Walker Kessler,SO,2022,Auburn,SEC,0.627,0.627,10.900,22.200,16.600,...,5.500,6.2,8.000,14.100,C,84.25,256.0,88.25,113.0,20.0
4,Mark Williams,SO,2022,Duke,ACC,0.700,0.702,13.633,20.800,17.400,...,4.833,7.2,4.433,11.567,C,84.00,242.4,90.50,117.0,29.0


**For whatever reason I can't find a dataset that has all the 2022 prospect birthdays, so I'm just going to manually make a list of them. I'll turn this list into a column as well as make a draft date column so that I can get the draft ages**

In [47]:
#create a new dataframe 
final = combined_with_rsci
#add draft date
final["Draft Date"] = "June 23, 2022"
#list of player birthdays
birthdays = ["August 19, 2000", "May 1, 2002", "May 10, 2001",
            "July 26, 2001", "December 16, 2001", "September 16, 2000",
            "December 18, 2000", "September 9, 2000", "March 27, 2001",
            "May 13, 2003", "July 10, 2000", "June 20, 2000",
            "September 1, 1999", "January 8, 2000", "September 23, 1999",
            "June 25, 1999", "April 25, 1998", "April 20, 2000",
            "April 17, 2001", "September 16, 2002", "February 27, 2002",
            "July 30, 2002", "October 20, 1999", "August 25, 2003",
            "June 19, 2002", "May 20, 2003", "July 12, 2002",
            "September 18, 2001", "June 29, 2002", "November 4, 2000",
            "June 29, 2001", "November 12, 2002", "May 11, 2003",
            "March 17, 2001", "January 16, 2000", "February 13, 2002",
            "November 18, 2003", "April 18, 2002", "March 7, 1999",
            "June 19, 1999", "December 3, 1999", "August 27, 1999",
            "April 14, 2001", "May 12, 2003", "May 26, 2000",
            "July 3, 2002", "October 3, 2002", "April 12, 2002",
            "December 31, 2002", "August 26, 2003", "November 27, 2002",
            "January 21, 2002", "December 15, 2001", "February 10, 2003",
            "November 8, 2002", "March 16, 2003", "September 11, 2002"]
#add birthday column
final["Birthday"] = np.array(birthdays)
#inspect data
final.tail()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking,Draft Date,Birthday
52,Aminu Mohammed,FR,2022,Georgetown,Big East,0.466,0.403,10.1,17.6,13.6,...,0.2,2.2,SG,75.50,212.8,83.00,104.0,24.0,"June 23, 2022","December 15, 2001"
53,Max Christie,FR,2022,Michigan State,Big Ten,0.494,0.449,3.4,9.2,6.4,...,1.1,1.9,SG,76.25,189.2,80.75,102.5,20.0,"June 23, 2022","February 10, 2003"
54,Bryce McGowens,FR,2022,Nebraska,Big Ten,0.533,0.453,2.4,14.6,8.7,...,-1.0,1.3,SG,77.25,181.2,80.75,103.5,28.0,"June 23, 2022","November 8, 2002"
55,Blake Wesley,FR,2022,Notre Dame,ACC,0.495,0.465,3.0,11.6,7.5,...,0.8,1.3,SG,75.00,187.2,81.25,103.0,100.0,"June 23, 2022","March 16, 2003"
56,Peyton Watson,FR,2022,UCLA,Pac-12,0.394,0.352,8.1,18.7,13.3,...,3.8,1.1,SF,78.75,203.4,84.50,107.5,11.0,"June 23, 2022","September 11, 2002"


In [48]:
final["Draft Date"] = pd.to_datetime(final["Draft Date"].str.replace(",", ""))
final["Birthday"] = pd.to_datetime(final["Birthday"].str.replace(",", ""))
#create a column with the draft age
final["Draft Age"] = final["Draft Date"] - final["Birthday"]
#this function changes the result from days to years...will be applied to the "Draft_Age" column
def convert_to_years(date):
    return round((date.total_seconds() / 60 / 60 / 24 / 365.25), 2)
#apply function to the column to get years
final["Draft Age"] = final["Draft Age"].apply(convert_to_years)
#drop the birth date and draft date columns since I now have the "Draft_Age" column
final = final.drop(columns=["Birthday", "Draft Date"])
#inspect the df
final.head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking,Draft Age
0,Keegan Murray,SO,2022,Iowa,Big Ten,0.621,0.594,9.800,19.933,14.867,...,10.3,2.933,13.233,PF,80.00,225.0,83.00,104.0,518.0,21.84
1,Chet Holmgren,FR,2022,Gonzaga,WCC,0.691,0.680,8.000,28.700,19.600,...,7.5,7.500,15.000,PF,84.00,195.0,90.00,115.0,1.0,20.15
2,Tari Eason,SO,2022,Louisiana State,SEC,0.615,0.559,10.500,19.800,15.200,...,9.0,5.700,14.700,PF,78.75,217.4,86.00,107.5,113.0,21.12
3,Walker Kessler,SO,2022,Auburn,SEC,0.627,0.627,10.900,22.200,16.600,...,6.2,8.000,14.100,C,84.25,256.0,88.25,113.0,20.0,20.91
4,Mark Williams,SO,2022,Duke,ACC,0.700,0.702,13.633,20.800,17.400,...,7.2,4.433,11.567,C,84.00,242.4,90.50,117.0,29.0,20.52


**The top 5 youngest and oldest players**

In [49]:
final.sort_values(by="Draft Age").head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking,Draft Age
36,Jalen Duren,FR,2022,Memphis,AAC,0.608,0.597,14.0,21.5,18.0,...,3.7,3.3,7.1,C,81.00,242.4,89.00,115.0,5.0,18.60
49,Trevor Keels,FR,2022,Duke,ACC,0.520,0.496,3.3,9.0,6.4,...,2.7,1.5,4.2,SG,75.25,223.8,79.25,101.0,23.0,18.83
23,AJ Griffin,FR,2022,Duke,ACC,0.630,0.613,3.9,13.7,9.1,...,6.6,1.5,8.1,SF,78.00,220.0,84.00,105.5,14.0,18.83
25,Jeremy Sochan,FR,2022,Baylor,Big 12,0.546,0.531,8.9,20.6,14.7,...,3.6,4.4,8.0,PF,81.00,230.0,84.00,108.5,156.0,19.09
9,Jabari Smith,FR,2022,Auburn,SEC,0.570,0.521,3.5,23.5,13.6,...,7.4,3.8,11.1,PF,82.00,220.0,85.00,110.0,6.0,19.11


In [50]:
final.sort_values(by="Draft Age", ascending = False).head()

,Player,Class,Season,School,Conf,TS%,eFG%,ORB%,DRB%,TRB%,...,OBPM,DBPM,BPM,Position,Height,Weight,Wingspan,Standing Reach,RSCI_Ranking,Draft Age
16,Bryson Williams,SR,2022,Texas Tech,Big 12,0.610,0.588,7.000,12.600,9.900,...,6.700,2.50,9.200,PF,80.00,235.0,86.25,107.0,272.0,24.16
38,Tyrese Martin,SR,2022,Connecticut,Big East,0.530,0.505,9.133,17.667,13.333,...,4.567,1.90,6.467,SG,77.25,215.0,80.00,103.5,156.0,23.30
39,Alondes Williams,SR,2022,Wake Forest,ACC,0.576,0.549,5.100,15.700,10.800,...,4.700,1.80,6.600,SG,75.75,209.0,79.00,102.0,272.0,23.01
15,Collin Gillespie,SR,2022,Villanova,Big East,0.591,0.546,1.532,10.790,6.213,...,5.632,2.31,7.942,PG,73.50,195.8,76.00,98.0,272.0,23.00
41,Jared Rhoden,SR,2022,Seton Hall,Big East,0.506,0.458,4.973,17.707,11.413,...,2.880,2.08,5.013,SG,78.00,210.0,82.50,104.0,148.0,22.82


**This last part will be arranging the columns in the proper order and with the proper names.**

In [51]:
#order of the columns
column_order = ["Player", "Position", "Season", "Draft Age", "Class", "School", "Height", "Weight", "Wingspan", 
                "Standing Reach", "RSCI_Ranking", "TS%", "eFG%", "ORB%", "DRB%", "TRB%", "AST%", "STL%", 
                "BLK%", "TOV%", "USG%", "PProd", "OBPM", "DBPM", "BPM"]
final = final[column_order]
#rename necessary columns
final = final.rename(columns = {"Season": "Draft Year", "RSCI_Ranking": "RSCI Ranking", "OBPM": "College OBPM", 
                                "DBPM": "College DBPM", "BPM": "College BPM"})
#inspect
final.head()

,Player,Position,Draft Year,Draft Age,Class,School,Height,Weight,Wingspan,Standing Reach,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,PProd,College OBPM,College DBPM,College BPM
0,Keegan Murray,PF,2022,21.84,SO,Iowa,80.00,225.0,83.00,104.0,...,14.867,8.467,2.400,6.667,6.967,25.967,554.000,10.3,2.933,13.233
1,Chet Holmgren,PF,2022,20.15,FR,Gonzaga,84.00,195.0,90.00,115.0,...,19.600,11.400,1.600,12.600,15.700,21.600,424.000,7.5,7.500,15.000
2,Tari Eason,PF,2022,21.12,SO,Louisiana State,78.75,217.4,86.00,107.5,...,15.200,10.100,4.500,6.200,13.800,31.800,506.000,9.0,5.700,14.700
3,Walker Kessler,C,2022,20.91,SO,Auburn,84.25,256.0,88.25,113.0,...,16.600,6.900,2.400,19.100,10.700,19.000,360.000,6.2,8.000,14.100
4,Mark Williams,C,2022,20.52,SO,Duke,84.00,242.4,90.50,117.0,...,17.400,7.900,1.533,10.800,11.700,19.333,323.333,7.2,4.433,11.567


In [52]:
#save dataframe to a csv named modeling_2022.csv
if not os.path.exists("modeling_2022.csv"):
    final.to_csv("modeling_2022.csv", index=False)
    print("Wrote dataframe containing all features for 2022 prospects to file called modeling_2022.csv")
else:
    print("File named modeling_2022.csv already exists")

Wrote dataframe containing all features for 2022 prospects to file called modeling_2022.csv
